In [1]:
## Importando a biblioteca
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [21]:
## Carregando o modelo treinado
modeldi=load_model('modeldi.h5')

## Carregando o encoder e o scaler
with open('onehot_encoder_smoking.pkl','rb') as file:
    onehot_encoder_smoking=pickle.load(file)

with open('label_encoder_genderdi.pkl','rb') as file:
    label_encoder_genderdi=pickle.load(file)

with open('scalerdi.pkl','rb') as file:
    scalerdi=pickle.load(file)

In [11]:
input_data={
    'gender': 'Male',
    'age': 42.0,
    'hypertension': 1,
    'heart_disease': 0,
    'smoking_history': 'never',
    'bmi': 27.32,
    'HbA1c_level': 6.6,
    'blood_glucose_level': 200
}

In [12]:
## Transformando os dados geography e gender para o código utilizado para treino da ANN
smoking_encoded=onehot_encoder_smoking.transform([[input_data['smoking_history']]]).toarray()
smoking_encoded_df=pd.DataFrame(smoking_encoded, columns=onehot_encoder_smoking.get_feature_names_out(['smoking_history']))
smoking_encoded_df

c:\Mestrado\env\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,smoking_history_No Info,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current
0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
## É necessário transformar a input data em um dataframe
input_df=pd.DataFrame([input_data])
input_df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level
0,Male,42.0,1,0,never,27.32,6.6,200


In [14]:
## Codificando a categorical variable
input_df['gender']=label_encoder_genderdi.transform(input_df['gender'])
input_df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level
0,1,42.0,1,0,never,27.32,6.6,200


In [15]:
## Agora, será substituída o geo encoded criado anteriormente na input data
input_df=pd.concat([input_df.drop("smoking_history",axis=1),smoking_encoded_df],axis=1)
input_df

,gender,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,smoking_history_No Info,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current
0,1,42.0,1,0,27.32,6.6,200,0.0,0.0,0.0,0.0,1.0,0.0


In [16]:
## Scaling the input data
input_scaled=scalerdi.transform(input_df)
input_scaled

array([[ 1.18538964e+00,  7.87021842e-04,  3.49272702e+00,
        -2.03558690e-01, -1.00746719e-03,  1.00269548e+00,
         1.52083822e+00, -7.47296245e-01, -3.18857974e-01,
        -2.04190583e-01, -3.21870310e-01,  1.36034024e+00,
        -2.62931169e-01]])

In [22]:
## Fazendo a predição
prediction=modeldi.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


array([[0.12896197]], dtype=float32)

In [23]:
## Verificando o resultado em probabilidade
prediction_proba=prediction[0][0]
prediction_proba

np.float32(0.12896197)

In [24]:
if prediction_proba > 0.5:
    print('Diabético')
else:
    print('Não Diabético')

Não Diabético
